In [ ]:
from retriever import create_vector_store

url = "https://docs.python.org/3/library/re.html"

retriever = create_vector_store(url = url, k = 10)

In [ ]:
from langchain_openai import ChatOpenAI

expt_llm = "gpt-4o"
llm = ChatOpenAI(model=expt_llm, max_tokens=4096)

In [30]:
from rag_agent import create_rag_agent

retrieval_grader = create_rag_agent(llm)

In [31]:
from code_agent import create_code_agent

code_gen_chain_oai = create_code_agent(llm)

In [33]:
from graph_state import GraphState


def format_docs(docs):
    return "\n\n\n".join(doc.page_content for doc in docs)

In [61]:
import subprocess


# Rag nodes
def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["messages"][0][1]
    test_cases = state["test_cases"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents[:], "test_cases": test_cases}

def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["messages"][0][1]
    error = state["error"]
    
    # If there was an error, add to the question
    if error == "yes":
        question += "\n\n"
        question += "\n\n".join([msg[1] for msg in state["messages"][-3:]])

    print(question)
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score["score"]
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}

### Nodes
def generate(state: GraphState):
    """
    Generate a code solution

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    error = state["error"]
    documents = state["documents"]

    concatenated_content = format_docs(documents)
    
    # We have been routed back to generation with an error
    if error == "yes":
        messages += [
            (
                "user",
                "Now, try again. Invoke the code tool to structure the output with a prefix, imports, and code block:",
            )
        ]

    # Solution
    code_solution = code_gen_chain_oai.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [
        (
            "assistant",
            f"{code_solution.prefix} \n Imports: {code_solution.imports} \n Code: {code_solution.code}",
        )
    ]

    # Increment
    iterations = iterations + 1
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


def code_check(state: GraphState):
    """
    Check code

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    print("---CHECKING CODE---")

    # State
    messages = state["messages"]
    code_solution = state["generation"]
    iterations = state["iterations"]
    test_cases = state["test_cases"]

    # Get solution components
    imports = code_solution.imports
    code = code_solution.code
    examples = test_cases

    # Check imports
    try:
        exec(imports)
    except Exception as e:
        print("---CODE IMPORT CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the import test: {str(e)}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # Save the code to a file called test_code.py
    file_name = 'test_code_w_examples.py'
    with open(file_name, 'w') as f:
        f.write(imports + "\n" + code + "\n" + examples)
    
    # Check execution

    # Run the script file using subprocess
    result = subprocess.run(['python', file_name], capture_output=True, text=True)
    
    # Check if there was an error
    if result.returncode != 0:
        print("An error occurred:")
        print(result.stderr)  # stderr will contain the traceback
        
        e = result.stderr
        print(f"Failed to run the script: {e}")
        print("---EXAMPLES BLOCK CHECK: FAILED---")

        error_message = [("user", f"Your solution failed the test case execution: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }


    # No errors
    print("---NO EXAMPLE TEST FAILURES---")
    return {
        "generation": code_solution,
        "messages": messages,
        "iterations": iterations,
        "error": "no",
    }


def reflect(state: GraphState):
    """
    Reflect on errors

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    code_solution = state["generation"]
    documents = state["documents"]

    concatenated_content = format_docs(documents)

    # Prompt reflection
    messages += [
            (
                "user",
                "Reflect on the previous error and code:",
            )
        ]

    # Add reflection
    reflections = code_gen_chain_oai.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [("assistant", f"Here are reflections on the error: {reflections}")]
    return {"generation": code_solution, "messages": messages, "iterations": iterations}

# Edges
def decide_to_finish(state: GraphState):
    """
    Determines whether to finish.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    error = state["error"]
    iterations = state["iterations"]

    if error == "no" or iterations == max_iterations:
        print("---DECISION: FINISH---")
        return "end"
    else:
        print("---DECISION: RE-TRY SOLUTION---")
        if flag == "reflect":
            return "reflect"
        else:
            return "generate"



In [76]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate", generate)  # generation solution
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("check_code", code_check)  # check code
workflow.add_node("reflect", reflect)  # reflect

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_edge("grade_documents", "generate")
workflow.add_edge("generate", "check_code")
workflow.add_conditional_edges(
    "check_code",
    decide_to_finish,
    {
        "end": END,
        "reflect": "reflect",
        "generate": "generate",
    },
)
workflow.add_edge("reflect", "retrieve")
app = workflow.compile()

In [ ]:
test_case_code

In [77]:
# Max tries
max_iterations = 10

from initial_prompt import create_test_case_code, create_question

# Reflect
flag = "reflect" # flag = 'reflect'

function_name = "remove_duplicate_phrases_or_sentences"
description = "Create an expression to remove duplicate sentences or phrases separated by punctuation."

test_case_data = [
        {"input": "This is a sentence. This is a sentence.", "expected": "This is a sentence."},
        {"input": "This is a sentence.This is a sentence.", "expected": "This is a sentence."},
        {"input": "Hello, hello, hello", "expected": "Hello,"},
        {"input": "a hard example . A hard example", "expected": "a hard example."}
    ]



test_case_code = create_test_case_code(function_name, test_case_data)

question = create_question(function_name, description, test_case_data)

In [78]:

solution = app.invoke({
    "messages": [("user", question)],
    "iterations": 0,
    "error":"", 
    "test_cases": test_case_code}, {"recursion_limit": 50})

---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
Create an expression to remove duplicate sentences or phrases seperated by punctuation.

Create a function called: remove_duplicate_phrases_or_sentences.

Test cases:
0. {"input": "This is a sentence. This is a sentence.", "expected": "This is a sentence."}
1. {"input": "This is a sentence.This is a sentence.", "expected": "This is a sentence."}
2. {"input": "Hello, hello, hello", "expected": "Hello,"}
3. {"input": "a hard example . A hard example", "expected": "a hard example."}
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
An error occurred:
Traceback (most recent call last):
  File "c:

In [79]:
!python test_code_w_examples.py

All test cases passed successfully.


In [80]:
solution

{'error': 'no',
 'messages': [('user',
   'Create an expression to remove duplicate sentences or phrases seperated by punctuation.\n\nCreate a function called: remove_duplicate_phrases_or_sentences.\n\nTest cases:\n0. {"input": "This is a sentence. This is a sentence.", "expected": "This is a sentence."}\n1. {"input": "This is a sentence.This is a sentence.", "expected": "This is a sentence."}\n2. {"input": "Hello, hello, hello", "expected": "Hello,"}\n3. {"input": "a hard example . A hard example", "expected": "a hard example."}'),
  ('assistant',
   "The problem is to remove duplicate sentences or phrases from a string where these sentences or phrases are separated by punctuation such as periods, commas, or spaces. The challenge is to use a regular expression to identify and remove the duplicates while keeping the first occurrence intact. The approach is to use a regular expression to identify the repeated phrases or sentences and then remove them. This can be done by capturing the p